In [ ]:
#Project website: https://www.kaggle.com/competitions/quora-insincere-questions-classification/data

In [1]:
import pandas as pd

In [2]:
df_ = pd.read_csv('train.csv')

In [3]:
df_

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [4]:
#Number of insincere questions are too low
# compared with other. Therefore, we 
# downsample sincere questions. 

df_1 = df_ [df_['target']==1]
df_0 = df_ [df_['target']==0]
df_00 = df_0.sample( len(df_1))

df = pd.concat([df_1,df_00])
df = df.sample(frac=1, random_state=42)  # 'frac=1' shuffles the entire DataFrame

# Reset the index
df = df.reset_index(drop=True)

In [5]:
df

,qid,question_text,target
0,827d8dc89d552fa9e8ff,How do I balance everything in life?,0
1,1183f8225b36e60f9c39,Submitting multiple letter's of recommendation...,0
2,913ec388c0a4c4871f90,How become master in seduction?,0
3,b7b7a8b1a451ea61ceb1,Are Alabama people different from everyone else?,1
4,2d2b2db58f795a0cc96b,"How do you use your ""night-owl"" or ""early-bird...",0
...,...,...,...
161615,576167efbdd3900d6770,How can you help someone be the best version o...,0
161616,cff482f84c48b2dd5a5e,Is it possible to chase your big dreams and ac...,0
161617,9b5da73167fbfb9137ac,Does moderate alcohol consumption directly inc...,0
161618,99c4e291bd3f7a2ae22b,What are the major cities in Cuba?,0


In [6]:
df = df.sample(10000)

In [7]:
df['target'].value_counts(normalize = True)

target
1    0.5048
0    0.4952
Name: proportion, dtype: float64

In [8]:
## TEXT PROCESSING

# kucuk harfe donustur
df["question_text"]=df["question_text"].str.lower()
# noktalama isaretlerini kaldirma
df["question_text"]=df["question_text"].str.replace("[^\w\s]","") 
# rakamlari kaldir
df["question_text"]=df["question_text"].str.replace("\d+","") 
# \newline ve enter\r kaldir
df["question_text"]=df["question_text"].str.replace("\n"," ").replace("\r","") 
# fazladan bosluklari azalt
df["question_text"]=df["question_text"].str.replace("  "," ")

In [9]:
# strip out html tags
import html
def del_html(text):
    return html.unescape(text)

# Apply to multiple columns
df['question_text'] = df['question_text'].apply(del_html)

In [10]:
#!pip install autocorrect

In [11]:
# FIND ROOTS, REMOVE PREFIXES, CORRECT SPELLING MISTAKES
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller
import re
from multiprocessing import Pool
import multiprocessing as mp

import concurrent.futures

def find_word_roots(text):
    
    spell = Speller(lang='en')
    
    # correct text
    #text = spell(text)
    pr = PorterStemmer()
    lm = WordNetLemmatizer()
    
    # find roots
    words = text.split(" ");
    kokler = [pr.stem(word) for word in words]
    
    #remove prefixes 
    kokler_k = [];
    for kok in kokler :
        kok=spell(kok)
        kokler_k.append( lm.lemmatize(kok,pos = 'v') )
        
    #remove     
    
    ret_str = "";
    for kok in kokler_k :
        ret_str = ret_str +" "+kok
        
    # remove https part
    ret_str =re.sub(r'https?\S+', '', ret_str)    
    #print(ret_str)    
    return ret_str


df['question_text'] = df['question_text'].apply(find_word_roots)    


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [13]:
x = df['question_text']
y = df['target']

In [14]:
vect =  CountVectorizer()
x = vect.fit_transform(x)

In [15]:
def algo_test(x,y):
#def algo_test(x_train,x_test,y_train,y_test):
    
    import numpy as np

    from scipy import stats
    import matplotlib.pyplot as plt
    %matplotlib inline 
    import seaborn as sns


    from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
    from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
    from sklearn.tree import ExtraTreeClassifier
    from sklearn.ensemble import GradientBoostingRegressor
    from xgboost import XGBRegressor 

    from sklearn.model_selection import train_test_split

    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    from sklearn.naive_bayes import GaussianNB
    from sklearn.naive_bayes import BernoulliNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import ExtraTreesClassifier
    from xgboost import XGBClassifier

    import warnings
    warnings.filterwarnings('ignore')
#plt.style.use('')

    
    
    G = GaussianNB()
    B = BernoulliNB()
    #K = KNeighborsClassifier()
    L = LogisticRegression()
    D = DecisionTreeClassifier()
    RF = RandomForestClassifier()
    ETC = ExtraTreesClassifier()
    XG = XGBClassifier()

    
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state = 13)
    
    algos = [G,B,L,D, RF, ETC, XG]
    #algos = [G,B,L,D,  RF, ETC]
    
    #algo_names = [ "Gaussian", "Bernoulli", "K-Neighbors", "Logistic Regression", "Decision Tree", "Random Forest", "ExtraTreeClassifier" ]
    algo_names = [ "Gaussian", "Bernoulli","Logistic Regression", "Decision Tree", \
                  "Random Forest", "ExtraTreeClassifier","XGBClassifier" ]
    
    ASC = [];
    
    result =  result = pd.DataFrame(columns =['Accuracy_Score'], index = algo_names)
    
    i = 0;
    for algo in algos :
        
        algo.fit(x_train,y_train)
        #print (accuracy_score(  y_test, algo.predict(x_test) ))
        ASC.append(  accuracy_score(  y_test, algo.predict(x_test) ) )
        
        print( "Confusion matrix: ",format(algo_names[i]) )
        i = i+1;
        print ( confusion_matrix(algo.predict(x_test),y_test) )
        #print (classification_report(y_test,algo.predict(x_test) ))
        
        #algo.fit(x,y)
        #print (accuracy_score(  y, algo.predict(x) ))
        #ASC.append(  accuracy_score(  y, algo.predict(x) ) )
    
    
    result.Accuracy_Score = ASC;

    return result.sort_values('Accuracy_Score', ascending=False)

In [16]:
algo_test(x.toarray(),y)

Confusion matrix:  Gaussian
[[569 133]
 [455 843]]
Confusion matrix:  Bernoulli
[[891 176]
 [133 800]]
Confusion matrix:  Logistic Regression
[[900 156]
 [124 820]]
Confusion matrix:  Decision Tree
[[794 220]
 [230 756]]
Confusion matrix:  Random Forest
[[833 161]
 [191 815]]
Confusion matrix:  ExtraTreeClassifier
[[847 140]
 [177 836]]
Confusion matrix:  XGBClassifier
[[883 170]
 [141 806]]


,Accuracy_Score
Logistic Regression,0.8600
Bernoulli,0.8455
XGBClassifier,0.8445
ExtraTreeClassifier,0.8415
Random Forest,0.8240
Decision Tree,0.7750
Gaussian,0.7060
